In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "veerendar2888" # username from the json file
os.environ['KAGGLE_KEY'] = "600e923d8c4f7b3a7241285ae4ed011f" # key from the json file
!kaggle competitions download -c new-york-city-taxi-fare-prediction # api copied from kaggle


100%|█████████████████████████████████████▊| 1.55G/1.56G [02:09<00:00, 15.8MB/s]
100%|██████████████████████████████████████| 1.56G/1.56G [02:09<00:00, 12.9MB/s]


In [ ]:
#!pip install kaggle

In [2]:
!unzip -o new-york-city-taxi-fare-prediction.zip -x train.csv

Archive:  new-york-city-taxi-fare-prediction.zip
  inflating: GCP-Coupons-Instructions.rtf  
  inflating: sample_submission.csv   
  inflating: test.csv                


In [18]:

# S3 prefix
prefix = 'kaggle-cat'
bucket = 'sklearn-script'

import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

In [6]:
import boto3
s3 = boto3.client('s3')
s3.upload_file(
    'train.csv', 'sklearn-script', 'train.csv')

In [23]:
WORK_DIRECTORY = 'data'
prefix =
train_input = sagemaker_session.upload_data('test.csv',
                                            bucket=bucket,
                                            key_prefix ="{}/{}".format(prefix,WORK_DIRECTORY) )

In [29]:
input_data = 's3://{}/{}/{}/train.csv'.format(bucket,prefix,WORK_DIRECTORY)

In [31]:
import pandas as pd
df = pd.read_csv(input_data)
df.shape

(300000, 25)

In [24]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor

region = boto3.session.Session().region_name

role = get_execution_role()
sklearn_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type='ml.t2.medium',
                                     instance_count=1)

In [ ]:
%%writefilw preprocessing.py

import argparse
import os
import warnings

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from haversine import haversine, Unit # mesuring distance (lat,lon)
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import sparse
from scipy.sparse import csr_matrix,hstack

from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore',category=DataConversionWarning)


def print_shape(df):
    
    print('Shape of train_x'.format(df.shape))
    
class OrdAttTransform(BaseEstimator,MetaEstimatorMixin):
    def __init__(self,ord_attr):
        self.ord_attr = ord_attr
        ord_dict = {'Freezing':1,'Cold':2,'Warm':3,'Hot':4,
                  'Boiling Hot':5,'Lava Hot':6,'Novice':1,
                  'Contributor':2,'Expert':3,'Master':4,'Grandmaster':5}
    def fit(self,X):
        return self
    def orde_scal(num_ordnl):
        seq = np.linspace(0,1,num_ordnl)
        return seq
    def transform(self,X):
    
        for attr in ord_attr:

            if attr in ('ord_1','ord_2'):
            X[attr] = X[attr].map(ord_dict)

            atlist = X[attr].unique().tolist()
            atlist.sort()
            arlist_len = len(atlist)
      
            attrmap = dict(zip(atlist, np.linspace(0,1,arlist_len)))
            X[attr] = X[attr].map(attrmap)
            return X


if __name__ == '__main__':
    
    parser = argparse.ArgumentParser()
    
    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    
    parser.add_argument('--train-test-split-ration',type=float,
                       default=0.3)
    args,_ = parser.parse_known_args()
    
    print('Received arguments  {}',format(args))
    
    input_data_path_train = os.path.join('/opt/ml/processing/input',
                                  'train.csv')
    
    input_data_path_test = os.path.join('/opt/ml/processing/input',
                                  'test.csv')
    
    print('Reading input data form {}',format(input_data_path_train))
    
    ## data processing
    ## load data
    train = pd.read_csv(input_data_path_train)
    test = pd.read_csv(input_data_path_test)
    
    # getting train test data id's
    train_id = train['id']
    test_id = test['id']
    target = train['target']
    
    # merging train test data sets into one data frame for feature engineering 
    df = pd.concat([train.drop(labels=['id','target'],axis=1), test.drop(labels='id',axis=1)])
    
    # features
    bin_attr = ['bin_3','bin_4']

    ord_dict = {'Freezing':1,'Cold':2,'Warm':3,'Hot':4,
                  'Boiling Hot':5,'Lava Hot':6,'Novice':1,
                  'Contributor':2,'Expert':3,'Master':4,'Grandmaster':5}

    ord_attr = ['ord_0','ord_1', 'ord_2', 'ord_3', 'ord_4', 'ord_5']

    nom_attr = ['nom_0','nom_1','nom_2','nom_3','nom_4','nom_5','nom_6','nom_7','nom_8','nom_9','day','month']
    
    
    class BinAttrTransform(BaseEstimator,MetaEstimatorMixin):
        def __init__(self,bin_attr):
            self.bin_attr = bin_attr
        def fit(self,X,y=None):
            return self
        def transform(self,X):
            bin_map = {'T':1,'F':0,'Y':1,'N':0}
            for att in bin_attr:
                X[att] = X[att].map(bin_map)
            return X
    
    
    